In [12]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
import datetime
import os
import pyarrow

In [22]:
dir_raw_data = "../data/raw_data"
dir_raw_data_movie_budget = "../data/raw_data/movie_budget"
dir_parsed_data = "../data/parsed_data"

t = datetime.date.today()
path_date = "day={}".format(datetime.datetime.strftime(t, "%Y%m%d"))
path_date = "day=20191009"
print(path_date)

url = "https://www.the-numbers.com/movie/budgets/all"

day=20191009


In [23]:
def request_data(page, url=url, save=True):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"
    }
    
    if page > 1:
        url += "/{}".format(str(page))
    
    req = Request(url, headers=headers)
    html = urlopen(req).read()
    
    df = pd.read_html(html, header=0, index_col=0)[0]
    if save:
        file_name = "movie_budget_{}".format(str(page))
        path_out = os.path.join(dir_raw_data_movie_budget, path_date, file_name)
        df.to_parquet(path_out, compression="snappy", engine="pyarrow")
    
    return df

In [24]:
page_start = 101
step = 100

In [25]:
data = request_data(1)
data_all = data
page = page_start
while data.shape[0] != 0:
    data = request_data(page)
    data_all = pd.concat([data_all, data])
    page += step
print(page-step)

5901


In [44]:
def rename_col(c):
    return c.replace(" ", "_")

def parse_amount(df, cols=["ProductionBudget", "DomesticGross", "WorldwideGross"]):
    for c in cols:
        df[c] = df[c].replace("[\$,\W]", "", regex=True).astype(float)
    return df

def parse_date(df, cols=["ReleaseDate"], renaming=True):
    df_out = df.copy()
    def parser(c):
        try:
            return datetime.datetime.strptime(c, "%b %d, %Y")
        except:
            return None
        
    for c in cols:
        df[c] = df[c].apply(lambda x: parser(x))
    return df

def get_movies_budget(df_budget):
    df = df_budget.copy()
    
    df = parse_amount(df)
    df = parse_date(df)
    
    d_renaming = {c: rename_col(c) for c in df.columns.tolist()}
    df_out = df.rename(columns=d_renaming)
    return df_out

In [45]:
path_movie_budget = os.path.join(dir_parsed_data, "movie_budget")

In [46]:
data_all.head()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
1,"Dec 17, 2009",Avatar,"$425,000,000","$760,507,625","$2,789,705,275"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,795,473,000"
4,"Apr 22, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 13, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [47]:
movie_budget = get_movies_budget(data_all)

In [48]:
movie_budget.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5850 entries, 1 to 5850
Data columns (total 5 columns):
ReleaseDate         5682 non-null datetime64[ns]
Movie               5850 non-null object
ProductionBudget    5850 non-null float64
DomesticGross       5850 non-null float64
WorldwideGross      5850 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 274.2+ KB


In [49]:
movie_budget.to_parquet(os.path.join(dir_parsed_data, "movie_budget"))

In [50]:
data = pd.read_parquet(os.path.join(dir_parsed_data, "movie_budget"))

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5850 entries, 1 to 5850
Data columns (total 5 columns):
ReleaseDate         5682 non-null datetime64[ns]
Movie               5850 non-null object
ProductionBudget    5850 non-null float64
DomesticGross       5850 non-null float64
WorldwideGross      5850 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 274.2+ KB


In [52]:
data.head()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
1,2009-12-17,Avatar,425000000.0,760507625.0,2.789705e+09
2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
3,2019-04-23,Avengers: Endgame,400000000.0,858373000.0,2.795473e+09
4,2015-04-22,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
5,2017-12-13,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09
